In [1]:
from pprint import pprint
import json
import os

import qcodes as qc

qc.set_mp_method('spawn')  # force Windows behavior on mac

qc.show_subprocess_widget()
from toymodel import AModel, MockGates, MockSource, MockMeter, AverageGetter, AverageAndRaw

model = AModel()
gates = MockGates('gates', model=model)
source = MockSource('source', model=model, server_name=None)
meter = MockMeter('meter', model=model)

station = qc.Station(gates, source, meter, meter.amplitude)
station.set_measurement(meter.amplitude)
c0, c1, c2, vsd = gates.chan0, gates.chan1, gates.chan2, source.amplitude


<IPython.core.display.Javascript object>

In [2]:
ll = qc.Loop(c0.sweep(0,1,0.2),0).each(qc.Loop(c0.sweep(5,6,0.1),0))
ll.run()
pprint(ll.snapshot())

DataSet: DataMode.PULL_FROM_SERVER, location='2016-04-29/01-12-52'
   chan0: chan0
   chan0_0: chan0
   amplitude: amplitude
started at 2016-04-29 01:12:53
{'active-loop': {'actions': [{'active-loop': {'actions': [{'__class__': 'qcodes.instrument.parameter.StandardParameter',
                                                           'instrument': 'MockMeter',
                                                           'label': 'Current '
                                                                    '(nA)',
                                                           'name': 'amplitude',
                                                           'ts': '2016-04-29 '
                                                                 '01:12:52',
                                                           'units': '',
                                                           'value': 0.117}],
                                              'delay': 0,
                                       

In [3]:
snap = ll.snapshot(update=True)

path = os.path.join(snap['data']['base_location'],
                    snap['data']['location'])

with open(path+'_snapshot.json', 'w') as meta_file:
    json.dump(snap, meta_file, sort_keys=True, indent=4, ensure_ascii=False)